In [1]:
import numpy as np
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import pickle
import os

In [2]:
from decoding_functions import *
from reading_data_functions import *
from spot_detection_functions import *

In [3]:
data_path = '/nfs/research1/gerstung/mg617/ISS_data/'

In [4]:
data_list = [
    'Exp_PD9694d2', #problem with registration; tried 1) matlab spots: EMSY has a weird pattern due to confusion with ACTB; 2) trackpy with radius 6: PR enhanced due to confusion with KRT8
    'Mut_PD9694d2', #problem with registration in top-right corner (wasn't able to resolve with trackpy search radius 6); carefull with the infeasible regions (barcode KCNT1mut and AP3B22mut seem overly expressed in this region for example); noise seems to high in rounds > 1 (only 17% of detected spots actually decoded when detected in matlab)
    'Mut_PD9694d3', #problem with registration
    'Mut_PD9694a2',
    'Mut_PD9694c2',
]

In [ ]:
spots_params = {'trackpy_diam_detect':5, 'trackpy_search_range':6, 'spot_diam_tophat':5} # parameters for spot detection: keep trackpy_serach_range small (3) and increase onlyto try to resolve registration issues
spots_params['trackpy_prc'] = 64 #by default this parameter is set to 64, decrease it to select more spots

In [ ]:
# Detect spots and decode them and save the results
for i in range(0, len(data_list)):

    dataset_name = data_list[i]
    print('loading ' + dataset_name)
    dataset_path = data_path + dataset_name + '/'
    tifs_path = dataset_path + 'selected_2k_tiles/'

    barcodes_01, K, R, C, gene_names, channels_info = read_taglist_and_channel_info_breastdata(dataset_path)

    tile_names = loadmat(dataset_path + 'decoding/tile_names.mat', squeeze_me=True)
    tiles_info = loadmat(dataset_path + 'selected_2k_tiles_DAPI_c01/tiles_info.mat', squeeze_me=True)
    tiles_info['filename_prefix'] = 'cycles_combined_opt_flow_registered_'

    tiles_to_load={'y_start':0, 'y_end':tiles_info['y_max'], 'x_start':0, 'x_end':tiles_info['x_max']} # select which tiles to load, including indices at the end
    
    # Load tile by tile and extract spots in each using trackpy (by default detects spots in each cycle and links them if in given search radius; call with anchors_cy_ind_for_spot_detect=0 to detect spots using only the 1st cycle)
    spots, spots_loc, _ = load_tiles_to_extract_spots(tifs_path, channels_info, C, R, tile_names, tiles_info, tiles_to_load, spots_params)
    # Save extracted spots
    extracted_spots_path = dataset_path + 'decoding/extracted-spots/'
    np.save(extracted_spots_path + dataset_name + '_spots_trackpy6.npy', spots)
    spots_loc.to_csv(extracted_spots_path + dataset_name + '_spots_trackpy6_locations.csv', index=False)

    print('decoding ' + dataset_name)
    # Estimate GMM parameters and compute the class probabilities 
    out = decoding_function(spots, barcodes_01)
    # Save decoding output
    with open(dataset_path + 'decoding/' + dataset_name + '_out.pickle', 'wb') as fp:
        pickle.dump(out, fp)

loading Exp_PD9694d2
Extracting spots from: X3_Y0 X4_Y0 X5_Y0 X6_Y0 X9_Y0 X10_Y0 X14_Y0 X15_Y0 X2_Y1 X3_Y1 X4_Y1 X5_Y1 X6_Y1 X7_Y1 X9_Y1 X10_Y1 X11_Y1 X12_Y1 X13_Y1 X14_Y1 X15_Y1 X16_Y1 X20_Y1 X1_Y2 X2_Y2 X3_Y2 X4_Y2 X5_Y2 X6_Y2 X7_Y2 X8_Y2 X9_Y2 X10_Y2 X11_Y2 X12_Y2 X13_Y2 X14_Y2 X15_Y2 X16_Y2 X18_Y2 X19_Y2 X20_Y2 X21_Y2 X22_Y2 X23_Y2 X24_Y2 X25_Y2 X1_Y3 X2_Y3 X3_Y3 X4_Y3 X5_Y3 X6_Y3 X7_Y3 X8_Y3 X9_Y3 X10_Y3 X11_Y3 X12_Y3 X13_Y3 X14_Y3 X15_Y3 X16_Y3 X17_Y3 X18_Y3 X19_Y3 X20_Y3 X21_Y3 X22_Y3 X23_Y3 X24_Y3 X25_Y3 X0_Y4 X1_Y4 X2_Y4 X3_Y4 X4_Y4 X5_Y4 X6_Y4 X7_Y4 X8_Y4 X9_Y4 X10_Y4 X11_Y4 X12_Y4 X13_Y4 X14_Y4 X15_Y4 X16_Y4 X17_Y4 X18_Y4 X19_Y4 X20_Y4 X21_Y4 X22_Y4 X23_Y4 X24_Y4 X25_Y4 X26_Y4 X0_Y5 X1_Y5 X2_Y5 X3_Y5 X4_Y5 X5_Y5 X6_Y5 X7_Y5 X8_Y5 X9_Y5 X10_Y5 X11_Y5 X12_Y5 X13_Y5 X14_Y5 X15_Y5 X16_Y5 X17_Y5 X18_Y5 X19_Y5 X20_Y5 X21_Y5 X22_Y5 X23_Y5 X24_Y5 X25_Y5 X26_Y5 X27_Y5 X0_Y6 X1_Y6 X2_Y6 X3_Y6 X4_Y6 X5_Y6 X6_Y6 X7_Y6 X8_Y6 X9_Y6 X10_Y6 X11_Y6 X12_Y6 X13_Y6 X14_Y6 X15_Y6 X16_Y6 X17_Y6 X

/homes/mg617/virtualEnvs/pytorch/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
100%|██████████| 60/60 [01:07<00:00,  1.13s/it]


loading Mut_PD9694d2
Extracting spots from: X15_Y0 X16_Y0 X5_Y1 X6_Y1 X7_Y1 X10_Y1 X11_Y1 X12_Y1 X13_Y1 X14_Y1 X15_Y1 X16_Y1 X17_Y1 X20_Y1 X21_Y1 X22_Y1 X23_Y1 X24_Y1 X26_Y1 X27_Y1 X2_Y2 X3_Y2 X4_Y2 X5_Y2 X6_Y2 X7_Y2 X8_Y2 X9_Y2 X10_Y2 X11_Y2 X12_Y2 X13_Y2 X14_Y2 X15_Y2 X16_Y2 X17_Y2 X18_Y2 X19_Y2 X20_Y2 X21_Y2 X22_Y2 X23_Y2 X24_Y2 X25_Y2 X26_Y2 X27_Y2 X28_Y2 X2_Y3 X3_Y3 X4_Y3 X5_Y3 X6_Y3 X7_Y3 X8_Y3 X9_Y3 X10_Y3 X11_Y3 X12_Y3 X13_Y3 X14_Y3 X15_Y3 X16_Y3 X17_Y3 X18_Y3 X19_Y3 X20_Y3 X21_Y3 X22_Y3 X23_Y3 X24_Y3 X25_Y3 X26_Y3 X27_Y3 X28_Y3 X1_Y4 X2_Y4 X3_Y4 X4_Y4 X5_Y4 X6_Y4 X7_Y4 X8_Y4 X9_Y4 X10_Y4 X11_Y4 X12_Y4 X13_Y4 X14_Y4 X15_Y4 X16_Y4 X17_Y4 X18_Y4 X19_Y4 X20_Y4 X21_Y4 X22_Y4 X23_Y4 X24_Y4 X25_Y4 X26_Y4 X27_Y4 X28_Y4 X0_Y5 X1_Y5 X2_Y5 X3_Y5 X4_Y5 X5_Y5 X6_Y5 X7_Y5 X8_Y5 X9_Y5 X10_Y5 X11_Y5 X12_Y5 X13_Y5 X14_Y5 X15_Y5 X16_Y5 X17_Y5 X18_Y5 X19_Y5 X20_Y5 X21_Y5 X22_Y5 X23_Y5 X24_Y5 X25_Y5 X26_Y5 X27_Y5 X28_Y5 X0_Y6 X1_Y6 X2_Y6 X3_Y6 X4_Y6 X5_Y6 X6_Y6 X7_Y6 X8_Y6 X9_Y6 X10_Y6 X11_Y6 X12

100%|██████████| 60/60 [00:47<00:00,  1.27it/s]


loading Mut_PD9694d3
Extracting spots from: X2_Y0 X3_Y0 X4_Y0 X14_Y0 X15_Y0 X1_Y1 X2_Y1 X3_Y1 X4_Y1 X5_Y1 X8_Y1 X9_Y1 X10_Y1 X11_Y1 X12_Y1 X13_Y1 X14_Y1 X15_Y1 X19_Y1 X20_Y1 X21_Y1 X1_Y2 X2_Y2 X3_Y2 X4_Y2 X5_Y2 X6_Y2 X7_Y2 X8_Y2 X9_Y2 X10_Y2 X11_Y2 X12_Y2 X13_Y2 X14_Y2 X15_Y2 X16_Y2 X17_Y2 X18_Y2 X19_Y2 X20_Y2 X21_Y2 X22_Y2 X23_Y2 X24_Y2 X25_Y2 X0_Y3 X1_Y3 X2_Y3 X3_Y3 X4_Y3 X5_Y3 X6_Y3 X7_Y3 X8_Y3 X9_Y3 X10_Y3 X11_Y3 X12_Y3 X13_Y3 X14_Y3 X15_Y3 X16_Y3 X17_Y3 X18_Y3 X19_Y3 X20_Y3 X21_Y3 X22_Y3 X23_Y3 X24_Y3 X25_Y3 X0_Y4 X1_Y4 X2_Y4 X3_Y4 X4_Y4 X5_Y4 X6_Y4 X7_Y4 X8_Y4 X9_Y4 X10_Y4 X11_Y4 X12_Y4 X13_Y4 X14_Y4 X15_Y4 